In [1]:
from bs4 import BeautifulSoup as bs
import os
import os.path
import re

RAW_HANSARD_DIR = 'scraped_content'

In [24]:
pq_paras = []
    
for file in os.listdir(RAW_HANSARD_DIR):
    filepath = os.path.join(RAW_HANSARD_DIR, file)
    if not os.path.isfile(filepath):
        continue
    
    print(f'\nProcessing: {file}')
    
    if file[0:4] == 'sprs':
        is_sprs = True
    else:
        is_sprs = False
    
    with open(filepath, 'r') as f:
        soup = bs(f, 'html.parser')
        
    if is_sprs:
        report_table = soup.find('div', {'class': 'reportTable'})
        report_table_rows = report_table.findChildren()

        # find out which rows correspond to pqs and which rows don't.
        for row in report_table_rows:
            paragraph = row.get_text()
            paragraph = paragraph.replace('\xa0', ' ')
            if not re.match(r'\d+.{8,75} asked the .+', paragraph):
                continue
            paragraph_no_num = paragraph[
                re.search('\d+', paragraph).span()[1]:]
            pq_paras.append(paragraph_no_num.strip())
    else:
        report_table = soup.find('div', {'class': 'reportTable'})
        raw_text = report_table.get_text()
        first_number_dot_occurrence = re.search(r'\d+\. .{8,75} asked the ', raw_text)
        
        if not first_number_dot_occurrence:
            continue

        # note: can't use honorifics as part of filtering criteria as the programmer doesn't have an exhaustive list
        # of honorifics and some of the honorifics are very obscure ones (e.g. Inche in Inche Rahamat Bin Kenap)
        raw_text = raw_text[first_number_dot_occurrence.span()[0]:]
        raw_text = raw_text.replace('\n\n', ' ')
        raw_paras = raw_text.split('\xa0\xa0\xa0\xa0')
        
        # i think honorific + name should take between 8 and 75 chars. 
        new_pq_paras = list(
            map(lambda s: s[re.search('\d+\.', s).span()[1]:].strip(),
            map(lambda s: re.sub('Column: \d+', '', s.replace('  ', ' ').strip()), 
            filter(lambda para: re.match(r'\d+\. .{8,75} asked the ', para), raw_paras))))
        
        pq_paras += new_pq_paras
        

    '''
    sit_date_str = soup.find('meta', {'name': 'Sit_Date'})['content']
    parl_no_str = soup.find('meta', {'name': 'Parl_No'})['content']
    '''
    
    
print(pq_paras)


Processing: sprs3topic_reportid=oral-answer-2802.html

Processing: sprs3topic_reportid=oral-answer-2806.html

Processing: sprs3topic_reportid=oral-answer-2809.html

Processing: sprs3topic_reportid=oral-answer-2812.html

Processing: sprs3topic_reportid=oral-answer-2818.html

Processing: sprs3topic_reportid=oral-answer-2819.html

Processing: sprs3topic_reportid=oral-answer-2822.html

Processing: sprs3topic_reportid=oral-answer-2825.html

Processing: sprs3topic_reportid=oral-answer-2828.html

Processing: sprs3topic_reportid=oral-answer-2833.html

Processing: sprs3topic_reportid=written-answer-10486.html

Processing: sprs3topic_reportid=written-answer-na-10315.html

Processing: sprs3topic_reportid=written-answer-na-10460.html

Processing: sprs3topic_reportid=written-answer-na-10587.html

Processing: sprs3topic_reportid=written-answer-na-10628.html

Processing: topic_reportid=003_19641116_S0003_T0011.html

Processing: topic_reportid=003_19641118_S0004_T0013.html

Processing: topic_reportid